In [1]:
#Customers who left are 1, current customers are 0
#Males are 0, females are 1
#might play around with using -1/1 instead

#LOOK UP KURTOSIS, THE GUY DID IT ON MONTHLY TENURE, LOOKED COOL, IDK


#NOT SURE IF I SHOULD ONEHOTENCODE OR JUST TOKENIZE AS 0-4 FOR CREDIT CARD TYPE
    #I went with tkenizing 0-4 for now


#Open-to-Buy: The difference between the credit limit assigned to a cardholder account and the present balance on the account.
    #DO RATIO BETWEEN THIS AND CREDIT LIMIT




In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
9451,712532058,Existing Customer,44,M,3,Post-Graduate,Married,$40K - $60K,Blue,36,...,12552.0,1405,11147.0,0.702,15508,95,0.792,0.112,0.000067,0.99993
1266,778137108,Existing Customer,43,M,4,High School,Single,$80K - $120K,Blue,37,...,30503.0,2517,27986.0,0.462,1860,47,0.306,0.083,0.000056,0.99994
5367,711243108,Existing Customer,50,M,1,Doctorate,Married,$80K - $120K,Blue,36,...,18964.0,2207,16757.0,0.625,3835,78,0.625,0.116,0.000411,0.99959
6484,779279883,Attrited Customer,60,F,2,High School,Married,Less than $40K,Blue,42,...,1438.3,0,1438.3,0.061,1628,42,0.077,0.000,0.994310,0.00569
9409,715051608,Attrited Customer,34,M,2,High School,Single,$60K - $80K,Blue,36,...,9719.0,2517,7202.0,0.906,9183,51,0.700,0.259,0.994310,0.00569


In [4]:
starter_df['Education_Level'].value_counts()

Graduate         2591
High School      1653
Uneducated       1204
College           844
Post-Graduate     431
Doctorate         358
Name: Education_Level, dtype: int64

In [5]:
#Shorten column names so they display better
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [6]:
#Drop unknowns
starter_df = starter_df[starter_df.Marital_Status != 'Unknown']
starter_df = starter_df[starter_df.Education != 'Unknown']
starter_df = starter_df[starter_df.Income != 'Unknown']

#NOT USING RIGHT NOWReplace M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': 0, 'F': 1}})

#Replace customer's status to 0 if they left and 1 if they stayed
starter_df = starter_df.replace({'Status' : {'Attrited Customer': 1, 'Existing Customer': 0}})





In [7]:
#use feature creation to make a ratio between dependents and income sources
#consider someone divorced as Single. only 10% of divorced people receive alimony payments
        #either a 1 (single/divorced) or a 2 (married)
        #divide that by the amount of dependents plus one, which represents how many people you take care of
                #i.e. a single person with no dependents only takes care of themself, so they are a 1. 
starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)

#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})


#Drop irrelevant column
starter_df = starter_df.drop(columns='CLIENTNUM')

In [8]:
#use feature creation to make a ratio between avg open to buy and credit limit

starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

In [9]:
#use feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [10]:
#There education categories are vague/ambiguous 
#In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    #8.9% had less than a high school diploma or equivalent.
        #our dataset shows 17% uneducated
    #27.9% had high school graduate as their highest level of school completed. 
        #our dataset shows 23.3%
    #14.9% had completed some college but not a degree.
        #ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    #10.5% had an associate degree as their highest level of school completed.
        #our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    #23.5% had a bachelor’s degree as their highest degree.
        #ours shows 11.9% for college
    #14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
        #ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError


counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])

,count,percentage
Graduate,2591,36.6%
High School,1653,23.3%
Uneducated,1204,17.0%
College,844,11.9%
Post-Graduate,431,6.1%
Doctorate,358,5.1%


In [11]:
#REMOVE GRADUATE

In [12]:
'''
It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, and dropping it will hurt our sample size. 
    The existence of a 'college' column suggests that graduate could mean beyond college. 
        Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
Its a high amount of the data (36.6%), so its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
With so little info on what Graduate means, I'm dropping it all. 
We could try to create a weighting system that roughly ranks the other columns based on years at school (?)
    Essentially:
        uneducated = 0, highschool=1, college=2, post-grad=3, doctorate=4

'''
#Change education to ranking system, drop graduate
starter_df = starter_df[starter_df.Education != 'Graduate']
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Post-Graduate' : 3, 'Doctorate' : 4}})



In [13]:
starter_df = starter_df.dropna()

In [14]:
#use OneHotEncoder for the income categories
enc = OneHotEncoder(sparse=False)
categorical_variables = ['Income']
encoded_data = enc.fit_transform(starter_df[categorical_variables])

encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

encoded_df = pd.concat(
    [
        starter_df,
        encoded_df
    ],
    axis=1
)
encoded_df.drop(columns=['Income'], inplace=True)
encoded_df = encoded_df.dropna()


c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
#Not positive I should be scaling both of these... need to come back and evaluate
scaler = StandardScaler()
encoded_df[['Tenure(month)', 'Total_Revolving_Bal']] = scaler.fit_transform(encoded_df[['Tenure(month)', 'Total_Revolving_Bal']])

In [17]:
#Remove columns that intimidate me

encoded_df.drop(columns=['Total_Relationship_Count', #¿que?
'Contacts_Count_12_mon', #Not a clue
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', #Some kind of wizardry, incantations
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', #I didnt make this and idk what it do or how it do it
'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', #Same as it ever was
],
inplace=True
)
encoded_df.sample(10)

,Status,Age,Gender,Education,Card Type,Tenure(month),Months_Inactive_12_mon,Total_Revolving_Bal,MaritalDependentRatio,CreditUsage,AvgTransValue,Income_$120K +,Income_$40K - $60K,Income_$60K - $80K,Income_$80K - $120K,Income_Less than $40K
2450,0.0,36.0,1.0,0.0,0.0,-0.672279,1.0,1.067240,0.25,0.62,45.88,0.0,0.0,0.0,0.0,1.0
2523,0.0,54.0,0.0,2.0,0.0,1.387384,1.0,0.885972,0.50,0.84,42.08,0.0,0.0,0.0,0.0,1.0
2683,0.0,33.0,1.0,3.0,0.0,-0.901131,2.0,-0.196565,0.25,0.42,59.25,0.0,0.0,0.0,0.0,1.0
3367,1.0,62.0,1.0,4.0,0.0,1.616236,3.0,-1.153610,0.50,0.89,92.83,1.0,0.0,0.0,0.0,0.0
3963,1.0,41.0,1.0,0.0,0.0,0.014275,2.0,1.623720,0.17,0.13,47.02,0.0,1.0,0.0,0.0,0.0
2608,0.0,55.0,0.0,2.0,1.0,1.044107,1.0,0.249633,0.17,0.96,63.42,0.0,0.0,0.0,0.0,1.0
1041,0.0,36.0,1.0,1.0,0.0,-1.129982,1.0,0.264844,0.25,0.67,62.21,0.0,0.0,1.0,0.0,0.0
3150,1.0,63.0,0.0,1.0,0.0,1.387384,3.0,-1.566850,0.50,1.00,39.24,0.0,0.0,0.0,0.0,1.0
3829,0.0,37.0,1.0,1.0,0.0,-2.045388,1.0,0.164703,0.10,0.32,69.55,0.0,0.0,0.0,0.0,1.0
2516,0.0,34.0,0.0,3.0,1.0,-0.901131,3.0,0.627380,0.08,0.95,38.43,0.0,1.0,0.0,0.0,0.0


In [18]:
y = encoded_df['Status']
X = encoded_df.drop(columns=['Status'])

In [20]:
#Check the value counts
y.value_counts()

0.0    1853
1.0     207
Name: Status, dtype: int64

In [21]:
#Split features and target in to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [22]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [23]:
#Choose a model

clf = LogisticRegression()
#clf = RandomForestClassifier()

In [24]:
#Fit the model on the training data
clf.fit(X_train, y_train)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [25]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [26]:
# Evaluate the model's performance
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 75.34%


In [27]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
4427,0.0,0.0
4451,1.0,1.0
4460,1.0,0.0
4469,0.0,0.0
4478,1.0,0.0


In [28]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[353 113]
 [ 14  35]]


In [29]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.96      0.76      0.85       466
         1.0       0.24      0.71      0.36        49

    accuracy                           0.75       515
   macro avg       0.60      0.74      0.60       515
weighted avg       0.89      0.75      0.80       515

